# Require imports

In [1]:
from __future__ import annotations
from typing import Dict
from collections.abc import Hashable
from itertools import combinations
from copy import deepcopy
from random import random, choice
import numpy
import miniball


def dist(vec_x: tuple[float, ...], vec_y: tuple[float, ...], p: float = 2) -> float:
    if len(vec_x) != len(vec_y):
        raise Exception("Vectors don't match.")
    total = 0
    for x, y in zip(vec_x, vec_y):
        total += abs(x - y) ** p
    return total ** (1 / p)


def generate_random_point_cloud(number_of_points: int, dimension: int = 3, lower_bound: float = 0,
                                upper_bound: float = 1, dp: int = -1) -> set[tuple[float, ...]]:
    if not number_of_points > 0:
        raise Exception("Number of points must be positive.")
    if not dimension > 0:
        raise Exception("Dimension must be positive.")
    if not lower_bound < upper_bound:
        raise Exception("Lower bound must be less than upper bound.")
    if not dp >= -1:
        raise Exception("dp must be -1 (no rounding) or non-negative.")
    bound_width = upper_bound - lower_bound
    point_cloud = set()
    for i in range(number_of_points):
        if dp == -1:
            point = tuple((random() * bound_width) + lower_bound for _ in range(dimension))
        else:
            point = tuple(round((random() * bound_width) + lower_bound, dp) for _ in range(dimension))

        point_cloud.add(point)
    return point_cloud

In [110]:
def __is_hashable__(obj: object):
    return isinstance(obj, Hashable)


def __is_valid_simplex__(simplex: set[object]):
    for vertex in simplex:
        if not __is_hashable__(vertex):
            return False
    return True


class AbstractSimplicialComplex:
    def __init__(self, vertices: set[object]):
        for vertex in vertices:
            if not __is_hashable__(vertex):
                raise Exception("Must have hashable types.")
        self.vertices = vertices
        self.simplices = {0: []}
        for vertex in self.vertices:
            self.simplices[0].append({vertex})

    def __str__(self):
        output = ["-" * 50, "\n"]
        max_simplex_count = 0
        for dim_simplices in self.simplices.values():
            if len(dim_simplices) > max_simplex_count:
                max_simplex_count = len(dim_simplices)
        if not max_simplex_count < 20:
            for dim, dim_simplices in sorted(self.simplices.items()):
                output.append(f"{str(dim)}: {str(len(dim_simplices))} simplices \n")
            output.append("-" * 50)
            output.append("\n\n")
            return "".join(output)

        for dim, dim_simplices in sorted(self.simplices.items()):
            output.append(f"{str(dim)}\n")
            for simplex in dim_simplices:
                output.append(f"{str(simplex)}\n")
            output[-1] = f"{output[len(output) - 1][:-2]}\n\n"
        output[-1] = output[-1][:-1]
        output.append("-" * 50)
        output.append("\n\n")
        return "".join(output)

    def __repr__(self):
        return self.__str__()

    def add_vertex(self, vertex: object) -> None:
        if vertex in self.vertices:
            return
        self.vertices.add(vertex)
        self.simplices[0].append({vertex})

    def add_simplex(self, simplex: set[object]) -> None:
        if not simplex.issubset(self.vertices):
            raise Exception("Simplex is not a subset of the vertices.")
        if len(simplex) == 0:
            raise Exception("Empty input.")
        if len(simplex) == 1:
            raise Exception("To add vertices, use add_vertex.")
        dim = len(simplex) - 1
        if dim - 1 not in self.simplices:
            raise Exception(
                "A face of this simplex is not in this complex, use force_add_simplex to force all faces in.")
        for vertex in simplex:
            face = simplex.difference({vertex})
            if face not in self.simplices[dim - 1]:
                raise Exception(
                    f"The face {face} of this simplex is not in this complex, use force_add_simplex to force all faces in.")
        if dim not in self.simplices:
            self.simplices[dim] = list()
        self.simplices[dim].append(simplex)

    def force_add_simplex(self, simplex: set[object]) -> None:
        if not __is_valid_simplex__(simplex):
            raise Exception("Simplex contains unhashable types.")
        dim = len(simplex) - 1
        if dim not in self.simplices:
            self.simplices[dim] = []
        if simplex in self.simplices[dim]:
            return
        for vertex in simplex:
            if vertex not in self.vertices:
                self.vertices.add(vertex)
                self.simplices[0].append({vertex})
        for l in range(2, len(simplex)):
            face_dim = l - 1
            if face_dim not in self.simplices:
                self.simplices[face_dim] = []
            for face_tuple in combinations(simplex, l):
                face = set(face_tuple)
                if face not in self.simplices[face_dim]:
                    self.simplices[face_dim].append(face)
        self.simplices[dim].append(simplex)

    def get_p_simplices(self, p: int) -> list[set[object]] | None:
        if p < 0:
            raise Exception("p must be non-negative.")
        if p not in self.simplices:
            return None
        return deepcopy(self.simplices[p])

    def get_vertices(self) -> set[object]:
        return self.vertices

# Cech complex

## Miniball

First we implement the Cech complex.

In [120]:
class FilteredCechComplex:
    def __init__(self, point_cloud: numpy.ndarray, dimension_limit: int | None = 5):
        self.dimension_limit = dimension_limit
        self.point_cloud = point_cloud
        self.vertices = set(map(tuple, point_cloud))
        if dimension_limit is None:
            dimension_limit = len(point_cloud)
        simplices_r = []
        for l in range(2, dimension_limit+1):
            for elem_tuple in combinations(point_cloud, l):
                _, r = miniball.get_bounding_ball(numpy.array(elem_tuple))
                simplex_r = {
                    "simplex": set(map(tuple, elem_tuple)),
                    "radius": r
                }
                simplices_r.append(simplex_r)
        self.filtration = sorted(simplices_r, key=lambda k: (round(k["radius"], 5), len(k["simplex"])))

    def get_filtration_depth(self) -> int:
        return len(self.filtration) + len(self.vertices)

    def get_complex(self, filtration_level: int) -> AbstractSimplicialComplex:
        if filtration_level == 0:
            return AbstractSimplicialComplex(set())
        if filtration_level < 0:
            raise ValueError("filtration_level must be non-negative.")
        filtration_depth = self.get_filtration_depth()
        if filtration_level > filtration_depth:
            raise ValueError(f"filtration_level is greater then the filtration depth of {filtration_depth}.")
        asc = AbstractSimplicialComplex(self.vertices)
        vertex_count = len(self.vertices)
        for i in range(filtration_level - vertex_count):
            simplex = self.filtration[i]["simplex"]
            asc.add_simplex(simplex)
        return asc

In [135]:
p_cloud = numpy.random.randn(10, 2)
f_complex = FilteredCechComplex(p_cloud, dimension_limit=None)

In [136]:
print(f_complex.get_complex(f_complex.get_filtration_depth()))

--------------------------------------------------
0: 10 simplices 
1: 45 simplices 
2: 120 simplices 
3: 210 simplices 
4: 252 simplices 
5: 210 simplices 
6: 120 simplices 
7: 45 simplices 
8: 10 simplices 
9: 1 simplices 
--------------------------------------------------


